In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../examples/florin/20181210_173702_meshID_1693d9a2-45c1-416b-8c7a-96ab881d1320.obj', 8)
driver=l.centralJoint()

mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [10, 7])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.show()

In [ ]:
l.saveVisualizationGeometry('deployed_meshID_1693d9a2-45c1-416b-8c7a-96ab881d1320.obj')
#l.writeLinkageDebugData('')

In [ ]:
#the stuff below will write a line model of the deployed stated
l.writeLinkageDebugData('deployed_1693d9a2-45c1-416b-8c7a-96ab881d1320.msh')
from linkage_utils import writeRodSegments
#the stuff below will write the polyline matching
writeRodSegments(l,'polyline_segments_pi3plus2pi12_1693d9a2-45c1-416b-8c7a-96ab881d1320.txt', zeroBasedIndexing=True)
#l.writeLinkageDebugData('')

In [ ]:
np.savetxt('restlen_meshID_1a219f3e-c853-4985-ad5f-f13b369c0e63.txt',l.getPerSegmentRestLength())

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(115) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);